In [1]:
from sklearn.datasets.samples_generator import make_blobs 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import normalize
from sklearn.model_selection import KFold
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy.linalg as LA
from scipy import linalg
import networkx as nx
import pandas as pd
import numpy as np
import random
import math
import sys, os
sys.path.append("..")
from llr import LLR 


In [8]:
n = 300
n_component = 100
p = 0.05
sigma = 0.1**2
_lambda = 0.1
deg = np.zeros(n)
D_inv = np.zeros((n_component, n_component))
component_count = 3
dim = 2

X, y = make_blobs(n_samples=n, n_features=dim, centers = [[0., 0.]], cluster_std=1.0)  
G_1 = nx.adjacency_matrix(nx.erdos_renyi_graph(n_component, p)).todense()
G_2 = nx.adjacency_matrix(nx.erdos_renyi_graph(n_component, p)).todense()
G_3 = nx.adjacency_matrix(nx.erdos_renyi_graph(n_component, p)).todense()

ro = [-1, 0, 1]
a_1 = np.mat(np.random.normal(ro[0], sigma, n_component))
a_2 = np.mat(np.random.normal(ro[1], sigma, n_component))
a_3 = np.mat(np.random.normal(ro[2], sigma, n_component))

mean = [0, 0]
cov = [[sigma, 0], [0, sigma]]  # diagonal covariance

"""
b_1 = np.random.multivariate_normal([-1, 0], cov, n)
b_2 = np.random.multivariate_normal([0, 0], cov, n)
b_3 = np.random.multivariate_normal([1, 0], cov, n)

"""
b_1 = np.mat(np.random.normal(ro[0], sigma, n_component))
b_2 = np.mat(np.random.normal(ro[1], sigma, n_component))
b_3 = np.mat(np.random.normal(ro[2], sigma, n_component))
b_4 = np.mat(np.random.normal(0, sigma, n_component))
b_5 = np.mat(np.random.normal(0, sigma, n_component))
b_6 = np.mat(np.random.normal(0, sigma, n_component))

G = [G_1, G_2, G_3]
a = np.block([[a_1.T], [a_2.T], [a_3.T]])
b = np.block([[b_1.T], [b_2.T], [b_3.T], [b_4.T], [b_5.T], [b_6.T]])
for j in range(0, component_count):
    try: 
        for i in range(0, n_component):
            deg[i+j*n_component] = np.sum(G[j][i,:]);
            if deg[i+j*n_component] == 0:
                raise ValueError()
            deg[i+j*n_component] = 1/deg[i+j*n_component];
    except ValueError:
        G[j] = nx.adjacency_matrix(nx.erdos_renyi_graph(n_component, p)).todense()
        i = 0
    
D = np.diag(deg)

G_adj = np.block([G[0], np.zeros((n_component, n_component*(component_count-1)))]);

for i in range(1, component_count-1):
    G_temp = np.block([np.zeros((n_component,(i)*n_component)), G[i], np.zeros((n_component, (component_count-i-1)*n_component))]);    
    G_adj = np.block([[G_adj], [G_temp]])
    
G_adj = np.block([[G_adj], [np.block([np.zeros((n_component, n_component*(component_count-1))), G[component_count-1]])]])

L = np.subtract(np.identity(n), np.matmul(D, G_adj)) 
norm_Lalpha = LA.norm(np.matmul(L, a))

M = np.block([L, np.zeros((n, n*(component_count-2)))]);
for i in range(1, component_count-2):
    M_temp = np.block([np.zeros((n,(i)*n)), L, np.zeros((n, (component_count-i-2)*n))]);    
    M = np.block([[M], [M_temp]])
    
M = np.block([[M], [np.block([np.zeros((n, n*(component_count-2))), L])]])

norm_Mbeta = LA.norm(np.matmul(M, b))
Diag_T = np.diag(X[:,0].T)
Diag = np.diag(X[:,0])
for i in range(1, dim): 
    Diag_T = np.block([[Diag_T], [np.diag(X[:,i].T)]])
    Diag = np.block([Diag, np.diag(X[:,i].T)])
    
vsu = LA.inv(np.matmul(Diag_T, Diag)+M)
mu = 1
P = LA.inv(np.identity(n) + mu*L - np.matmul(Diag,np.matmul(vsu, Diag_T)))

row_norms_X = np.zeros(n)
for i in range(0, n): 
    row_norms_X[i] = LA.norm(X[i])

max_row_norm_X = max(row_norms_X)

def bias_alpha(lambda_val):
    bias_alpha_value = norm_Lalpha**2*mu**2/gamma**2 + norm_Mbeta*v**2*max_row_norm_X**2/(gamma**2*lambda_val**2)
    print(bias_alpha_value)
    return bias_alpha_value

def bias_beta(lambda_val):
    bias_beta_value = bias_alpha(lambda_val)**2*max_row_norm_X**2/lambda_val**2 + norm_Mbeta*v**2/(lambda_val**2)
    print(bias_beta_value)
    return bias_beta_value


In [ ]:
"""""
norm_sq_sum = 0

for j in range(0, component_count):
    try: 
        for i in range(0, n_component):
            D[i,i] = np.sum(G[j][i,:]);
            if D[i,i] == 0:
                raise ValueError()
            D_inv[i,i] = 1/D[i,i];
    except ValueError:
        G[j] = np.random.normal(ro[j], sigma, 100)
    print(j)    
    

    L = np.subtract(np.identity(n_component), np.matmul(D_inv, G[j]))
    norm_sq_sum = norm_sq_sum + np.matmul(np.matmul(L,a[j]), np.matmul(L,a[j]).T)
    print(norm_sq_sum)

norm_L = math.sqrt(norm_sq_sum)
print(norm_L)
"""